In [ ]:
from google import genai
from google.genai import types


# Load the API key from a file
# in our case the file is untracked on git to avoid leaking the key
with open("api_key.txt", "r") as f:
    api_key = f.read().strip()

client = genai.Client(api_key=api_key)

stock = 'GME'


response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=f'''
                You need to act as an expert linguist, trader and researcher, read the reddit comment about {stock} that is given to you, and classify wether the comment is bullish or bearish about {stock}.
                Your answer should be 1 if the comment is positive about the stock, -1 if it is negative, and 0 if it is neutral. Do not reply to the comment, just classify it.
                '''),
    contents='I think GME is going up despite the market being bearish'
    )

print(response.text)

1



In [17]:
posts_path = "../historical_submissions.parquet"
comments_path = "../historical_comments.parquet"

# requires: pyarrow and fastparquet
posts = pd.read_parquet(posts_path)
comments = pd.read_parquet(comments_path)

In [21]:
comments_list = list(comments['body'])
comments_list

['INTC Intel breaking out',
 'BlackBerry already climbed. need to wait for a dip',
 'zom jagx ears heavy volume',
 'CCTV n lucid motors',
 'Here’s a low effort comment for a low effort post.',
 '$GNRS',
 'is vldr good stock?',
 'VLDR is a Lidar stock which is used in Electric Vehicles. So good for long term. Its on consolidation phase  right now. on watch',
 'Waiting for big news next week.',
 'Looks like Market is bullish. Are we going to expect any pullback this Friday or early next week ?',
 'all ev space is bullish. but market is volatile. if you are long term then no worry else use stop loss',
 'hello everyone',
 'I have a $100, what stock can I buy today?',
 'people were buying Expr $10.64 , AMC $15.64, \n\nTrivago is coming up TRVG 4.76 \nCIDM 1.76\n\nSPAC \nCCIV $22',
 'Is boeing a buy?',
 'for long term yes',
 'KODK with 25.58% short interest. Ready to fly 🚀',
 "CLVS and its shareholders have invested a lot of money in developing medicines to cure people with CANCER. And becau

In [16]:
from mlx_lm import load, generate

model, tokenizer = load("mlx-community/Llama-3.2-3B-Instruct-4bit")

stock = 'GME'

messages = [{
     'role': 'system',
     'content': f'''
                You need to act as an expert linguist, trader and researcher, read the reddit comment about {stock} that is given to you, and classify wether the comment is bullish or bearish about {stock}.
                Your answer should be 1 if the comment is positive about the stock, -1 if it is negative, and 0 if it is neutral. Do not reply to the comment, just classify it.
                ''',
   },
   {
     'role': 'user',
     'content': 'I think GME is going up despite the market being bearish',
   }]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True
)

text = generate(model, tokenizer, prompt=prompt, verbose=True)

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 96791.63it/s]


0
Prompt: 132 tokens, 355.723 tokens-per-sec
Generation: 2 tokens, 12.702 tokens-per-sec
Peak memory: 3.615 GB
